In [1]:
import pandas as pd
import os
import pickle

In [5]:
with open(os.path.dirname(os.getcwd()) + "/conversions/area_to_country.pkl", "rb") as file:
    area_to_country = pickle.load(file)

print(area_to_country)

    Area                                          Countries
0     21  [Republic of Korea, Latvia, Lithuania, Greenla...
1     27  [Republic of Korea, Morocco, Latvia, Lithuania...
2     31  [Republic of Korea, Haiti, Jamaica, Lithuania,...
3     34  [Republic of Korea, Argentina, Morocco, Latvia...
4     37  [Republic of Korea, Morocco, Türkiye, Japan, G...
5     41  [Republic of Korea, Argentina, Latvia, Lithuan...
6     47  [Republic of Korea, Latvia, Lithuania, Japan, ...
7     51  [Republic of Korea, Sri Lanka, Djibouti, Lithu...
8     57  [Bangladesh, Republic of Korea, Sri Lanka, Jap...
9     61  [China, Macao SAR, Russian Federation, Republi...
10    67  [Bulgaria, Russian Federation, Republic of Kor...
11    71  [Republic of Korea, Guam, Viet Nam, Japan, Wal...
12    77  [Republic of Korea, Lithuania, Japan, Nicaragu...
13    81  [Republic of Korea, Latvia, Lithuania, Japan, ...
14    87  [Republic of Korea, Argentina, Latvia, Lithuan...


In [50]:
file_path = os.path.dirname(os.getcwd()) + "/raw_data/"

cpi_fab = pd.read_csv(file_path + "cpi_food_and_beverage.csv")
gdp_pc = pd.read_csv(file_path + "gdp_per_capita.csv")

In [52]:
with open("countries_list.pkl", "rb") as file:
    fish_countries = pickle.load(file)

econ_countries = gdp_pc["COUNTRY.Name"].drop_duplicates()

In [54]:
with open("countries_mapping_rev.pkl", "rb") as file:
    countries_mapping_rev = pickle.load(file)

In [55]:
def map_countries(country_list):
    return [countries_mapping_rev[country] for country in country_list if country in countries_mapping_rev]

area_to_country["Countries"] = area_to_country["Countries"].apply(map_countries)

In [45]:
gdp_pc2 = pd.read_csv(file_path + "gdp_per_capita2.csv")

print(gdp_pc2.columns)

Index(['DATASET', 'SERIES_CODE', 'OBS_MEASURE', 'COUNTRY.Name',
       'INDICATOR.Name', 'FREQUENCY.Name', 'SCALE.Name', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021'],
      dtype='object')


In [56]:
area_gdp = {}

cols = [
    "2004",
    "2005",
    "2006",
    "2007",
    "2008",
    "2009",
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
]

for area, countries in zip(area_to_country["Area"], area_to_country["Countries"]):
    area_gdp[area] = pd.DataFrame([[0] * len(cols)], columns=cols)
    count = 0
    for c in countries:
        gdp_c = gdp_pc2[gdp_pc2["COUNTRY.Name"] == c].reset_index()[cols]
        if not gdp_c.isna().any().any():
            area_gdp[area] += gdp_c
            count += 1
    area_gdp[area] /= count

In [92]:
def aggregate_econ_data(
    data,
    cols=[
        "2004",
        "2005",
        "2006",
        "2007",
        "2008",
        "2009",
        "2010",
        "2011",
        "2012",
        "2013",
        "2014",
        "2015",
        "2016",
        "2017",
        "2018",
        "2019",
        "2020",
        "2021",
    ],
    area_to_country=area_to_country
):
    area_econ = {}

    for area, countries in zip(area_to_country["Area"], area_to_country["Countries"]):
        area_econ[area] = pd.DataFrame([[0] * len(cols)], columns=cols)
        count = 0
        for c in countries:
            temp = data[data["COUNTRY.Name"] == c].reset_index()[cols]
            print(type(temp.iloc[0]["2004"]))
            if not (temp.isna().any().any() or len(temp) == 0):
                area_econ[area] += temp
                count += 1
        area_econ[area] /= count
        area_econ[area] = area_econ[area].iloc[0].to_dict()

    return area_econ    

In [85]:
aggregate_econ_data(cpi_fab)

{21: {'2004': 336.9063734311022,
  '2005': 349.29988179485764,
  '2006': 366.6760553659619,
  '2007': 408.20526295780326,
  '2008': 469.8659564883596,
  '2009': 470.89152778142767,
  '2010': 470.6700928636387,
  '2011': 500.25662517297917,
  '2012': 515.5048499051989,
  '2013': 530.4027131973319,
  '2014': 528.9311324030258,
  '2015': 534.4836589052672,
  '2016': 535.3397391000531,
  '2017': 553.2746594554175,
  '2018': 565.1967605931134,
  '2019': 591.2712905071173,
  '2020': 620.4654571084201,
  '2021': 637.9870664824581},
 27: {'2004': 325.9399102579015,
  '2005': 336.3641829336943,
  '2006': 352.1827842754243,
  '2007': 390.3459931609659,
  '2008': 447.752708364327,
  '2009': 447.9041556583622,
  '2010': 447.61371556950155,
  '2011': 474.85536686852555,
  '2012': 488.94336256853404,
  '2013': 502.7317478034166,
  '2014': 499.667287469733,
  '2015': 504.16627149805316,
  '2016': 504.94312246437033,
  '2017': 521.7178882834467,
  '2018': 532.605922154578,
  '2019': 556.6166729519098,

In [88]:
import pprint
pprint.pprint(countries_mapping_rev)

{'Afghanistan, Islamic Republic of': 'Afghanistan',
 'Albania': 'Albania',
 'Algeria': 'Algeria',
 'Andorra, Principality of': 'Andorra',
 'Angola': 'Angola',
 'Antigua and Barbuda': 'Antigua and Barbuda',
 'Argentina': 'Argentina',
 'Armenia, Republic of': 'Armenia',
 'Aruba, Kingdom of the Netherlands': 'Aruba',
 'Australia': 'Australia',
 'Austria': 'Austria',
 'Azerbaijan, Republic of': 'Azerbaijan',
 'Bahamas, The': 'Bahamas',
 'Bahrain, Kingdom of': 'Bahrain',
 'Bangladesh': 'Bangladesh',
 'Barbados': 'Barbados',
 'Belarus, Republic of': 'Belarus',
 'Belgium': 'Belgium',
 'Belize': 'Belize',
 'Benin': 'Benin',
 'Bhutan': 'Bhutan',
 'Bolivia': 'Bolivia (Plurinational State of)',
 'Bosnia and Herzegovina': 'Bosnia and Herzegovina',
 'Botswana': 'Botswana',
 'Brazil': 'Brazil',
 'Brunei Darussalam': 'Brunei Darussalam',
 'Bulgaria': 'Bulgaria',
 'Burkina Faso': 'Burkina Faso',
 'Burundi': 'Burundi',
 'Cabo Verde': 'Cabo Verde',
 'Cambodia': 'Cambodia',
 'Cameroon': 'Cameroon',
 'Can

In [93]:
fish_ex = pd.read_csv(file_path + "fish_exports.csv")

aggregate_econ_data(fish_ex)

<class 'str'>


TypeError: unsupported operand type(s) for +: 'int' and 'str'